In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import os
import re
import matplotlib.pyplot as plt

tqdm.pandas()

In [2]:
df = pd.read_pickle("data/pickles/total_listings_viz.pkl")

In [5]:
negh_list = df["neighbourhood_cleansed"].tolist()
neigh_counter = {x:negh_list.count(x) for x in negh_list}

In [10]:
neigh_counter = dict(sorted(neigh_counter.items(), key=lambda item: item[1], reverse=True))

In [11]:
neigh_counter

{'I Centro Storico': 14593,
 'Centro Storico': 7971,
 'VII San Giovanni/Cinecittà': 2121,
 'II Parioli/Nomentano': 1926,
 'XIII Aurelia': 1775,
 'BUENOS AIRES - VENEZIA': 1604,
 'Santo Stefano': 1564,
 'San Lorenzo': 1545,
 'XII Monte Verde': 1479,
 'Castello': 1435,
 'Cannaregio': 1380,
 'Porto - Saragozza': 1376,
 'V Prenestino/Centocelle': 1274,
 'DUOMO': 1254,
 'Bergamo': 1186,
 'Campo di Marte': 1127,
 'SARPI': 875,
 'Rifredi': 850,
 'San Marco': 846,
 'BRERA': 844,
 'LORETO': 834,
 'Pendino': 831,
 'CENTRALE': 823,
 'NAVIGLI': 795,
 'VIII Appia Antica': 789,
 'TICINESE': 784,
 'San Ferdinando': 759,
 'X Ostia/Acilia': 730,
 'ISOLA': 646,
 'Montecalvario': 639,
 'Chiaia': 632,
 'Piave 1860': 615,
 'San Polo': 584,
 'XIV Monte Mario': 566,
 'XXII MARZO': 563,
 'Dorsoduro': 552,
 'San Donato - San Vitale': 546,
 'Santa Croce': 536,
 'Avvocata': 518,
 'XI Arvalia/Portuense': 515,
 'Navile': 480,
 'San Giuseppe': 476,
 'Isolotto Legnaia': 473,
 'IV Tiburtina': 461,
 'III Monte Sacro':

In [ ]:
## Run the following to compute the mappings for host locations
#!python3 -m scripts.mappings_host_location

## Load Visualization Pipeline for all the periods
## Insert one of the following arguments to run the visualization:
## - sep
## - dic
## - mar
## - jun

#!python3 -m scripts.pipeline_viz sep
#!python3 -m scripts.pipeline_viz dic
#!python3 -m scripts.pipeline_viz mar
#!python3 -m scripts.pipeline_viz jun

#!python3 -m scripts.pipeline_ML sep
#!python3 -m scripts.pipeline_ML dic
#!python3 -m scripts.pipeline_ML mar
#!python3 -m scripts.pipeline_ML jun